<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Generación de texto usando caracteres y redes recurrentes </center></span>

<center>Introducción</center>

<figure>
<center>
<img src="../Imagenes/text_generation_sampling.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Muestreo  de palabras</p>
</figcaption>
</figure>

Fuente: [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation)

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Referencias</span>

1. [Tensorflow, Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation)
1. [Ralf C. Staudemeyer and Eric Rothstein Morris,Understanding LSTM a tutorial into Long Short-Term Memory Recurrent Neural Networks*, arxiv, September 2019](https://arxiv.org/pdf/1909.09586.pdf)
1. [Karpathy, The Unreasonable Effectiveness of Recurrent Neural Networks](  http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
1. [Proyecto Gutemberg](https://www.gutenberg.org/)

##   <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Importa módulos requeridos](#Importa-módulos-requeridos)
* [Lee los datos](#Descarga-los-datos)
* [Crea  el alfabeto](#Crea-el-alfabeto)
* [Crea los diccionarios ](#Crea-los-diccionarios )
* [Transforma el texto en un arreglo de caracteres](#Transforma-el-texto-en-un-arreglo-de-caracteres)
* [Creación de datos de entrenamiento y etiquetas](#Creación-de-datos-de-entrenamiento-y-etiquetas)
* [Construcción del Modelo](#Construcción-del-Modelo)
* [Prueba del modelo ](#Prueba-del-modelo)
* [Entrenamiento](#Entrenamiento)
* [Generación de texto](#Generación-de-texto)
* [Entrenamiento personalizado](#Entrenamiento-personalizado)

##   <span style="color:blue">Introducción</span>

Este cuaderno es una adaptación del tutorial de tensorflow [Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation).

Se muestra cómo generar texto usando un RNR basado en caracteres. Trabajaremos con un conjunto de datos de los algunos documentos de cuentos dsiponibles en la biblioteca libre Gutemberg, con los códigos:  

+ 55514-0.txt, 
+ 61244-0.txt, 
+ pg36805.txt, 
+ pg45438.txt, 
+ pg46000.txt, 
+ 30053-0.txt,

Y los poemas del Daniel Montenegro, uno de los autores

+ Poemas_Output.txt
+ Poemas_Todo.txt

Los datos fueron preparados por Alvaro Montenegro, uno de los autores.


Dada una secuencia de caracteres a partir de estos datos  se entrena un modelo para predecir el siguiente carácter en la secuencia. 


Se pueden generar secuencias de texto más largas llamando al modelo repetidamente.


##   <span style="color:blue">Importa módulos requeridos</span>

In [51]:
import tensorflow as tf

import numpy as np
import os
import time
import glob

print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.4.1


##   <span style="color:blue">Lee los datos</span>

Obtiene el path en donde están los datos y los lee en una única lista.

In [65]:
path = '../Datos/'
files = glob.glob(path+ '*.txt')
files

['../Datos/55514-0.txt',
 '../Datos/Poemas_Output.txt',
 '../Datos/pg46000.txt',
 '../Datos/Poemas_Todo.txt',
 '../Datos/pg45438.txt',
 '../Datos/pg36805.txt',
 '../Datos/30053-0.txt',
 '../Datos/61244-0.txt']

In [108]:
text = []
for file in files:
    t = open(files[6], 'rb').read().decode(encoding='utf-8')
    text.extend(t)


In [109]:
len(text)#691080


686344

### Una mirada a los primeros 250  caracters

In [68]:
# Echa un vistazo a los primeros 250 caracteres del texto
print(text[:250])

['\r', '\n', 'E', 'L', ' ', 'P', 'Á', 'J', 'A', 'R', 'O', ' ', 'E', 'N', ' ', 'L', 'A', ' ', 'N', 'I', 'E', 'V', 'E', '\r', '\n', '\r', '\n', '\r', '\n', 'E', 'r', 'a', ' ', 'c', 'i', 'e', 'g', 'o', ' ', 'd', 'e', ' ', 'n', 'a', 'c', 'i', 'm', 'i', 'e', 'n', 't', 'o', '.', ' ', 'L', 'e', ' ', 'h', 'a', 'b', 'í', 'a', 'n', ' ', 'e', 'n', 's', 'e', 'ñ', 'a', 'd', 'o', ' ', 'l', 'o', ' ', 'ú', 'n', 'i', 'c', 'o', ' ', 'q', 'u', 'e', ' ', 'l', 'o', 's', ' ', 'c', 'i', 'e', 'g', 'o', 's', '\r', '\n', 's', 'u', 'e', 'l', 'e', 'n', ' ', 'a', 'p', 'r', 'e', 'n', 'd', 'e', 'r', ',', ' ', 'l', 'a', ' ', 'm', 'ú', 's', 'i', 'c', 'a', ';', ' ', 'y', ' ', 'f', 'u', 'e', ' ', 'e', 'n', ' ', 'e', 's', 't', 'e', ' ', 'a', 'r', 't', 'e', ' ', 'm', 'u', 'y', ' ', 'a', 'v', 'e', 'n', 't', 'a', 'j', 'a', 'd', 'o', '.', ' ', 'S', 'u', ' ', 'm', 'a', 'd', 'r', 'e', '\r', '\n', 'm', 'u', 'r', 'i', 'ó', ' ', 'p', 'o', 'c', 'o', 's', ' ', 'a', 'ñ', 'o', 's', ' ', 'd', 'e', 's', 'p', 'u', 'é', 's', ' ', 'd', 'e

In [69]:
text[:50]

['\r',
 '\n',
 'E',
 'L',
 ' ',
 'P',
 'Á',
 'J',
 'A',
 'R',
 'O',
 ' ',
 'E',
 'N',
 ' ',
 'L',
 'A',
 ' ',
 'N',
 'I',
 'E',
 'V',
 'E',
 '\r',
 '\n',
 '\r',
 '\n',
 '\r',
 '\n',
 'E',
 'r',
 'a',
 ' ',
 'c',
 'i',
 'e',
 'g',
 'o',
 ' ',
 'd',
 'e',
 ' ',
 'n',
 'a',
 'c',
 'i',
 'm',
 'i',
 'e',
 'n']

##   <span style="color:blue">Crea  el alfabeto</span>

Este alfabeto es una lista que contiene las letras mayúscula y munúsculas, y algunos caracteres especiales, incluyendo el salto de línea '\n'.

In [110]:
# Los caracteres únicos en el archivo.
vocab = sorted(set(text))
print ('{} caracteres únicos'.format(len(vocab)))

72 caracteres únicos


In [111]:
vocab

['\n',
 '\r',
 ' ',
 '!',
 '"',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '¡',
 '´',
 '¸',
 'Á',
 'Ñ',
 'á',
 'é',
 'í',
 'ñ',
 'ó',
 'ú']

##   <span style="color:blue">Crea los diccionarios </span>

- caracter a índice
- índice a caracter

In [112]:
# Creación de un mapeo de caracteres únicos a índices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [113]:
char2idx

{'\n': 0,
 '\r': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '(': 5,
 ')': 6,
 '*': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 ':': 11,
 ';': 12,
 '?': 13,
 'A': 14,
 'B': 15,
 'C': 16,
 'D': 17,
 'E': 18,
 'F': 19,
 'G': 20,
 'H': 21,
 'I': 22,
 'J': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'Y': 35,
 'a': 36,
 'b': 37,
 'c': 38,
 'd': 39,
 'e': 40,
 'f': 41,
 'g': 42,
 'h': 43,
 'i': 44,
 'j': 45,
 'k': 46,
 'l': 47,
 'm': 48,
 'n': 49,
 'o': 50,
 'p': 51,
 'q': 52,
 'r': 53,
 's': 54,
 't': 55,
 'u': 56,
 'v': 57,
 'x': 58,
 'y': 59,
 'z': 60,
 '¡': 61,
 '´': 62,
 '¸': 63,
 'Á': 64,
 'Ñ': 65,
 'á': 66,
 'é': 67,
 'í': 68,
 'ñ': 69,
 'ó': 70,
 'ú': 71}

In [78]:
idx2char 

array(['\n', '\r', ' ', '!', '"', '(', ')', '*', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B',
       'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P',
       'Q', 'R', 'S', 'T', 'U', 'V', 'Y', '[', ']', 'a', 'b', 'c', 'd',
       'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q',
       'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '´', '¸', 'Á', 'Ñ',
       'á', 'é', 'í', 'ñ', 'ó', 'ú'], dtype='<U1')

##   <span style="color:blue">Transforma el texto en un arreglo de caracteres </span>

In [114]:
text_as_int = np.array([char2idx[c] for c in text])

In [115]:
text_as_int

array([ 1,  0, 18, ...,  0,  1,  0])

In [81]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '(' :   5,
  ')' :   6,
  '*' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '0' :  11,
  '1' :  12,
  '2' :  13,
  '3' :  14,
  '4' :  15,
  '5' :  16,
  '6' :  17,
  '7' :  18,
  '8' :  19,
  ...
}


### Demostración del uso de los diccionarios

La función *repr* convierte el argumento en una expresión imprimible(cuando es posible).

In [116]:
# Muestre cómo los primeros 13 caracteres del texto se asignan a números enteros
print ('{} ---- caracteres mapeados a int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

['\r', '\n', 'E', 'L', ' ', 'P', 'Á', 'J', 'A', 'R', 'O', ' ', 'E'] ---- caracteres mapeados a int ---- > [ 1  0 18 24  2 28 64 23 14 30 27  2 18]


### La tarea de predicción


Dado un caracter, o una secuencia de caracteres, ¿cuál es el próximo caracter más probable? Esta es la tarea en la que vamos a entrenar al modelo. 


La entrada al modelo será una secuencia de caracteres, y entrenamos al modelo para predecir la salida, el siguiente carácter en cada paso de tiempo.

Dado que los RNR mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres calculados hasta este momento, ¿cuál es el siguiente carácter?


##   <span style="color:blue">Creación de datos de entrenamiento y etiquetas  </span>

El texto se divide  en secuencias de entrenamiento. Cada secuencia de entrada contendrá  una longitud *seq_length* de caracteres del texto.

Para cada secuencia de entrada, las etiquetas correspondientes contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Así que primero se divide el texto en trozos de *seq_length + 1*. Por ejemplo, digamos que *seq_length* es 3 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de destino "ola".


In [117]:
# La oración de longitud máxima que queremos para una sola entrada en caracteres
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Luego se usa la función *tf.data.Dataset.from_tensor_slices* para convertir el vector de texto en una secuencia de índices de caracteres. Un tensor de enteros.

In [118]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])




E
L
 
P
Á
J
A
R


In [64]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

El método por lotes nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.


In [119]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\r\nEL PÁJARO EN LA NIEVE\r\n\r\n\r\nEra ciego de nacimiento. Le habían enseñado lo único que los ciegos\r\nsue'
'len aprender, la música; y fue en este arte muy aventajado. Su madre\r\nmurió pocos años después de dar'
'le la vida; su padre, músico mayor de un\r\nregimiento, hacía un año solamente. Tenía un hermano en Amé'
'rica que no\r\ndaba cuenta de sí; sin embargo, sabía por referencias que estaba casado,\r\nque tenía dos '
'niños muy hermosos y ocupaba buena posición. El padre\r\nindignado, mientras vivió, de la ingratitud de'


In [88]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [120]:
for item in sequences.take(5):
  print(repr(idx2char[item.numpy()]))

array(['\r', '\n', 'E', 'L', ' ', 'P', 'Á', 'J', 'A', 'R', 'O', ' ', 'E',
       'N', ' ', 'L', 'A', ' ', 'N', 'I', 'E', 'V', 'E', '\r', '\n', '\r',
       '\n', '\r', '\n', 'E', 'r', 'a', ' ', 'c', 'i', 'e', 'g', 'o', ' ',
       'd', 'e', ' ', 'n', 'a', 'c', 'i', 'm', 'i', 'e', 'n', 't', 'o',
       '.', ' ', 'L', 'e', ' ', 'h', 'a', 'b', 'í', 'a', 'n', ' ', 'e',
       'n', 's', 'e', 'ñ', 'a', 'd', 'o', ' ', 'l', 'o', ' ', 'ú', 'n',
       'i', 'c', 'o', ' ', 'q', 'u', 'e', ' ', 'l', 'o', 's', ' ', 'c',
       'i', 'e', 'g', 'o', 's', '\r', '\n', 's', 'u', 'e'], dtype='<U1')
array(['l', 'e', 'n', ' ', 'a', 'p', 'r', 'e', 'n', 'd', 'e', 'r', ',',
       ' ', 'l', 'a', ' ', 'm', 'ú', 's', 'i', 'c', 'a', ';', ' ', 'y',
       ' ', 'f', 'u', 'e', ' ', 'e', 'n', ' ', 'e', 's', 't', 'e', ' ',
       'a', 'r', 't', 'e', ' ', 'm', 'u', 'y', ' ', 'a', 'v', 'e', 'n',
       't', 'a', 'j', 'a', 'd', 'o', '.', ' ', 'S', 'u', ' ', 'm', 'a',
       'd', 'r', 'e', '\r', '\n', 'm', 'u', 'r', 'i', '


Para cada secuencia, duplíquela y cámbiela para formar el texto de entrada y de destino utilizando el método  map para aplicar una función simple a cada lote. Es  similar a la función apply de R.

In [121]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [122]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [123]:
dataset.take(2)

<TakeDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [124]:
for item in dataset.take(2):
  print(item)

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 1,  0, 18, 24,  2, 28, 64, 23, 14, 30, 27,  2, 18, 26,  2, 24, 14,
        2, 26, 22, 18, 34, 18,  1,  0,  1,  0,  1,  0, 18, 53, 36,  2, 38,
       44, 40, 42, 50,  2, 39, 40,  2, 49, 36, 38, 44, 48, 44, 40, 49, 55,
       50, 10,  2, 24, 40,  2, 43, 36, 37, 68, 36, 49,  2, 40, 49, 54, 40,
       69, 36, 39, 50,  2, 47, 50,  2, 71, 49, 44, 38, 50,  2, 52, 56, 40,
        2, 47, 50, 54,  2, 38, 44, 40, 42, 50, 54,  1,  0, 54, 56])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 0, 18, 24,  2, 28, 64, 23, 14, 30, 27,  2, 18, 26,  2, 24, 14,  2,
       26, 22, 18, 34, 18,  1,  0,  1,  0,  1,  0, 18, 53, 36,  2, 38, 44,
       40, 42, 50,  2, 39, 40,  2, 49, 36, 38, 44, 48, 44, 40, 49, 55, 50,
       10,  2, 24, 40,  2, 43, 36, 37, 68, 36, 49,  2, 40, 49, 54, 40, 69,
       36, 39, 50,  2, 47, 50,  2, 71, 49, 44, 38, 50,  2, 52, 56, 40,  2,
       47, 50, 54,  2, 38, 44, 40, 42, 50, 54,  1,  0, 54, 56, 40])>)
(<tf.Tensor: shap

In [125]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    

Input data:  '\r\nEL PÁJARO EN LA NIEVE\r\n\r\n\r\nEra ciego de nacimiento. Le habían enseñado lo único que los ciegos\r\nsu'
Target data: '\nEL PÁJARO EN LA NIEVE\r\n\r\n\r\nEra ciego de nacimiento. Le habían enseñado lo único que los ciegos\r\nsue'


In [126]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\r')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 18 ('E')
Step    2
  input: 18 ('E')
  expected output: 24 ('L')
Step    3
  input: 24 ('L')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 28 ('P')


### Creación de lotes de entrenamiento

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, necesitamos mezclar los datos y empaquetarlos en lotes.

In [127]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

##   <span style="color:blue">Construcción del Modelo  </span>


Use `tf.keras.Sequential` para definir el modelo. Para este sencillo ejemplo, se utilizan tres capas para definir nuestro modelo:

In [128]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [129]:
vocab_size

72

In [130]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [131]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


<figure>
<center>
<img src="../Imagenes/text_generation_training.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de generación de texto</p>
</figcaption>
</figure>

Fuente: [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation)

##   <span style="color:blue">Prueba del modelo  </span>


Ahora ejecute el modelo para ver que se comporta como se esperaba.

Primero verifique la forma de la salida:

In [132]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 72) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de secuencia de la entrada es `100` pero el modelo puede ejecutarse en entradas de cualquier longitud.

El dataset de entrenamiento que tiene lotes de tamaño 64 * 100, los cuales tiene orden aleatorio. El primer lote actual y se pasa por el modelo.

El modelo tiene de momento los pesos iniciales. El lote de datos pasa por el modelo y devuelve un tensor de tamaño (64, 100, 84).

Como el vocabulario tiene 84 caracteres y cada sequencia tiene tamaño 100, el modelo predice el siguiente caracterer para cada caracter en la entrada.

La predicción funciona así:

Por cada caracter de entrada, se predice el siguiente caracter así: el modelo asigna un valor numérico a cada elemento en el vocabulario. Si se eligiera el máximo el caracter seleccionado sería simplemente el carater con el valor mas grande.

Ya por fuera del modelo.

El siguiente es el vector de predicciones del siguiente caracter para todos los caracteres de la primera secuencia en el primer bloque.

In [133]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           18432     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 72)            73800     
Total params: 4,030,536
Trainable params: 4,030,536
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones reales del modelo, necesitamos tomar muestras de la distribución de salida, para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de los caracteres.

### Nota 

Es mejor muestrear  esta distribución,  que tomar el *argmax* para evitar  que el modelo quede atascado en un bucle. Probemos para el primer ejemplo en el lote:

In [134]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 72), dtype=float32, numpy=
array([[ 6.25738129e-03,  5.26489178e-03, -3.67547339e-03, ...,
        -6.28688326e-03,  9.09127994e-04, -6.80384971e-03],
       [-1.13937622e-02,  4.77731228e-04, -2.16814363e-03, ...,
         1.32553745e-02,  1.34873260e-02, -7.78970961e-03],
       [-8.15382041e-03,  8.70817155e-03, -1.33496802e-03, ...,
         1.41692981e-02,  1.76555272e-02, -2.85920594e-03],
       ...,
       [-6.19465671e-03,  2.93969573e-03,  5.07240836e-03, ...,
         6.39514066e-03, -8.57854262e-04,  1.86331011e-02],
       [-2.42418610e-05, -9.15902667e-04,  7.22392369e-03, ...,
        -4.23473399e-03, -2.86005484e-03,  1.04329251e-02],
       [-2.64084293e-03, -2.22867588e-03,  9.56252776e-03, ...,
        -3.35635175e-03,  9.54606012e-03,  3.16632190e-03]], dtype=float32)>

Estos valores son logits para obtener probabilidades. Entonces se normaliza a la una distribución categórica, es decir se usa softmax. Con esto, se genera una muestra de la variable categórica para seleccionar el caracter.

El siguiente es el vector de predicciones del siguiente caracter para todos los caracteres de  la primera secuencia en el primer bloque.

Vamos a predecir los caracteres como indicamos.

In [135]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[23],
       [58],
       [17],
       [ 6],
       [12],
       [17],
       [63],
       [60],
       [21],
       [27],
       [31],
       [70],
       [ 5],
       [53],
       [52],
       [26],
       [50],
       [61],
       [39],
       [43],
       [ 6],
       [13],
       [16],
       [41],
       [54],
       [ 7],
       [64],
       [23],
       [22],
       [70],
       [44],
       [70],
       [17],
       [30],
       [32],
       [35],
       [62],
       [68],
       [52],
       [ 1],
       [29],
       [31],
       [71],
       [14],
       [22],
       [32],
       [63],
       [ 2],
       [18],
       [32],
       [18],
       [48],
       [ 2],
       [52],
       [ 5],
       [22],
       [16],
       [55],
       [11],
       [69],
       [60],
       [59],
       [55],
       [11],
       [37],
       [29],
       [55],
       [ 5],
       [38],
       [29],
       [60],
       [17],
       [44],
   

In [136]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([23, 58, 17,  6, 12, 17, 63, 60, 21, 27, 31, 70,  5, 53, 52, 26, 50,
       61, 39, 43,  6, 13, 16, 41, 54,  7, 64, 23, 22, 70, 44, 70, 17, 30,
       32, 35, 62, 68, 52,  1, 29, 31, 71, 14, 22, 32, 63,  2, 18, 32, 18,
       48,  2, 52,  5, 22, 16, 55, 11, 69, 60, 59, 55, 11, 37, 29, 55,  5,
       38, 29, 60, 17, 44, 15, 59, 64, 54,  3, 14, 63, 25, 14, 63,  0, 63,
       30, 36, 20, 23, 63, 43, 23, 59, 49, 29, 23, 32, 37, 68, 51])

Esto nos da, en cada paso de tiempo, una predicción del siguiente índice de caracteres:

In [137]:
sampled_indices

array([23, 58, 17,  6, 12, 17, 63, 60, 21, 27, 31, 70,  5, 53, 52, 26, 50,
       61, 39, 43,  6, 13, 16, 41, 54,  7, 64, 23, 22, 70, 44, 70, 17, 30,
       32, 35, 62, 68, 52,  1, 29, 31, 71, 14, 22, 32, 63,  2, 18, 32, 18,
       48,  2, 52,  5, 22, 16, 55, 11, 69, 60, 59, 55, 11, 37, 29, 55,  5,
       38, 29, 60, 17, 44, 15, 59, 64, 54,  3, 14, 63, 25, 14, 63,  0, 63,
       30, 36, 20, 23, 63, 43, 23, 59, 49, 29, 23, 32, 37, 68, 51])

Esta salida significa que el caracter predicho como el siguiente para el primer caracter es el caracter 58 y asi...

Los textos lucen así actualmente

Decodifíquelos para ver el texto predicho por este modelo no entrenado:

In [138]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' su seno hacia el firmamento\r\nun número infinito de pequeñas columnas de humo, las cuales al\r\nextend'

Next Char Predictions: 
 'JxD);D¸zHOSó(rqNo¡dh)?Cfs*ÁJIóióDRTY´íq\rQSúAIT¸ ETEm q(ICt:ñzyt:bQt(cQzDiByÁs!A¸MA¸\n¸RaGJ¸hJynQJTbíp'


##   <span style="color:blue">Entrenamiento </span>

En este punto, el problema puede tratarse como un problema de clasificación estándar. Dado el estado RNN anterior, y la entrada en este paso de tiempo, predice la clase del siguiente carácter.

Se define una función de pérdida adecuada para este caso. Esta función recibe los targets (labels) y las predicciones La función calcula la distribución a partir de las predicciones y genera para cada caracter un valor predicho. Esto es lo que se necesita para calcular la entropía cruzada en este caso (promedio). Este es el valor retornado. En el ejemplo, como entra una secuencia y se predicen 100 caracteres hay 100 distribuciones para el siguiente caracter. Una por cada caracter en la entrada. En total hay un lote de 64 secuencias. Luego la función de pérdida se basa en el promedio de  64∗100=6400  entropías cruzadas dispersas.

### Adjuntar un optimizador y una función de pérdida

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits, debemos establecer el indicador `from_logits`.

In [139]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 72)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.2749677



Configure el procedimiento de entrenamiento utilizando el método `tf.keras.Model.compile`. Usaremos `tf.keras.optimizers.Adam` con argumentos predeterminados y la función de pérdida.

In [140]:
model.compile(optimizer='adam', loss=loss)

### Configuración de checkpoints

Utilice un `tf.keras.callbacks.ModelCheckpoint` para asegurarse de que los puntos de control se guarden durante el entrenamiento. Se crea un directorio en el cual se guardará checkpoints con el estado del modelos. Los pesos son guardados allí para uso posterior

In [141]:
# Directorio en donde se guardarán los checkpoints
checkpoint_dir = './training_checkpoints'
# Nombre de los archivos de los  checkpoint 
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Ejecuta el entrenamiento

In [142]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

En history queda al información de loss, y acurracy para revisión.

##   <span style="color:blue">Generación de texto </span>

### Restaura el último  checkpoint

En esta sección vamos a tomar un pre-entranamiento. 

Usaremos los pesos almacenados, pero ahora vamos a cambiar el modelo para recibir lotes de tamaño 1. Es decir vamos a recibir una secuencia para hacer la predicción a partir de esa secuencia.


Debido a la forma en que se pasa el estado RNN de un paso a otro, el modelo solo acepta un tamaño de lote fijo una vez construido.

Para ejecutar el modelo con un tamaño de lote diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el punto de control.

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

### El bucle de predicción

El siguiente bloque de código genera el texto:

* Comienza eligiendo una cadena de inicio, inicializando el estado de la  RNR y configurando el número de caracteres a generar.

* Obtenga la distribución de predicción del siguiente carácter utilizando la cadena de inicio y el estado de la  RNR.

* Luego, use una distribución categórica para calcular el índice del carácter predicho. Use este caracter predicho como nuestra próxima entrada al modelo.

* El estado RNN devuelto por el modelo se retroalimenta al modelo para que ahora tenga más contexto, en lugar de una sola palabra. Después de predecir la siguiente palabra, los estados RNN modificados se retroalimentan nuevamente en el modelo, que es cómo aprende a medida que obtiene más contexto de las palabras predichas previamente.

La siguiente es la linea que define la capa GRU arriba en la definición del modelo.

In [ ]:
#tf.keras.layers.GRU(rnn_units,  return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')

El parámetro *stateful* determina si los valores de estado recurrente deben matenerse (True) o no, cuando se pasa al siguiente caracter. En este caso, al mantenerse esos valores, se mantiene la memoria de la secuencia inicial y de los caracteres que van siendo generados. 

Por eso, solamente se pasa en el primer paso la secuencia de entrada completa. Desde el segundo paso solamente se pasa el nuevo caracter (que es el predicho). Con esta predicción y la memoria del resto de la secuencia anterior mantenida en estado recurrente se hace la siguiente predicción y así sucesivamente tomado la última secuencia pasada por la capa.

Para pasar la última secuencia tratada en la capa GRU, se usa el parámetro *return_sequences*. Por defecto es falso, es decir se omite en la salida la última secuencia pasada por la capa GRU. Aquí hemos colocado *return_sequences=True*, para pasar únicamente el último carater predicho.




Al observar el texto generado, verá que el modelo sabe cuándo colocar mayúsculas, hacer párrafos e imita un vocabulario de escritura similar a Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar oraciones coherentes.

<figure>
<center>
<img src="../Imagenes/text_generation_sampling.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Muestreo  de palabras</p>
</figcaption>
</figure>

Fuente: [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation)

La siguiente función implementa el ciclo de predicción

In [ ]:
def generate_text(model, start_string):
    # Paso de evaluación (generación de texto usando el modelo aprendido)

    # Número de caracteres a generar
    num_generate = 1000

    # Convertir nuestra cadena de inicio en números (vectorizar)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Cadena vacía para almacenar nuestros resultados
    text_generated = []

    # Las bajas temperaturas dan como resultado un texto más predecible.
    # Las temperaturas más altas dan como resultado un texto más sorprendente.
    # Experimente para encontrar la mejor configuración.
    temperature = 1.0

    # Aquí batch size == 1
    model.reset_states() # borra las unidades recurrentes
    for i in range(num_generate):
        predictions = model(input_eval)
        # eliminar la dimensión del lote
        predictions = tf.squeeze(predictions, 0)

        # usar una distribución categórica para predecir la palabra devuelta por el modelo
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pasamos la palabra predicha como siguiente entrada al modelo
        # junto con el estado oculto anterior
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

### Notas.

1. *reset_states* borra solo los estados recurrentes de la red. Vale la pena mencionar que dependiendo de si la opción *stateful = True* se configuró en la red, el comportamiento de esta función podría ser diferente. Si no está configurado, todos los estados se restablecen automáticamente después de cada cálculo por lotes en su red (por ejemplo, después de llamar a *fit, predict y evaluate* también). Si no es así, debe llamar a reset_states cada vez que desee que las llamadas de modelo consecutivas sean independientes.
2. *tf.expand_dims* devuelve un tensor con una dimensión adicional insertada en el eje de índice.

In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))

Lo más fácil que puedes hacer para mejorar los resultados es entrenarlo por más tiempo (prueba `EPOCHS = 30`).

También puede experimentar con una cadena de inicio diferente, o intentar agregar otra capa RNN para mejorar la precisión del modelo, o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

##  <span style="color:blue">Entrenamiento personalizado </span>



El procedimiento de entrenamiento anterior es simple, pero no le da mucho control.

Entonces, ahora que ha visto cómo ejecutar el modelo manualmente, descomprimimos el ciclo de entrenamiento e implementémoslo nosotros mismos. Esto proporciona un punto de partida si, por ejemplo, se implementa _ aprendizaje curricular_ para ayudar a estabilizar la salida de bucle abierto del modelo.

Usaremos `tf.GradientTape` para rastrear los gradientes. Puede obtener más información sobre este enfoque leyendo el [eager execution guide](https://www.tensorflow.org/guide/eager).

El procedimiento funciona de la siguiente manera:

* Primero, inicialice el estado RNN. Hacemos esto llamando al método `tf.keras.Model.reset_states`.

* Luego, repita el conjunto de datos (lote por lote) y calcule las * predicciones * asociadas con cada una.

* Abra un `tf.GradientTape`, y calcule las predicciones y pérdidas en ese contexto.

* Calcular los gradientes de la pérdida con respecto a las variables del modelo utilizando el método `tf.GradientTape.grads`.

* Finalmente, dé un paso hacia abajo utilizando el método `tf.train.Optimizer.apply_gradients` del optimizador.

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [ ]:
# Paso de entrenamiento
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # inicializando el estado oculto al comienzo de cada época
    # inicialmente oculto es Ninguno
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

    if batch_n % 100 == 0:
        template = 'Epoch {} Batch {} Loss {}'
        print(template.format(epoch+1, batch_n, loss))
 
    # guardar (punto de control) el modelo cada 5 épocas
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

- [Regresar al inicio](Contenido)